## 머신러닝 실습

### 텍스트 마이닝

#### 개념

##### 텍스트 마이닝
- 비정형 텍스트 데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석 기법 이용
- 프로세스
    - 텍스트 전처리 > 특성 벡터화 > 머신러닝 모델 구축, 학습, 평가
    - 텍스트 전처리
        - 토큰화, 불용어 제거, 표제어 추출, 형태소 분석

##### 특성 벡터화 / 추출
- 컴퓨터는 글을 읽을 수 없음 -> 단어 기반으로 특성 추출, 숫자형 벡터값으로 표현
-  BoW, Word2ve 라이브러리(모듈) 존재

##### LDA
- 문서에 잠재되어 있는 토픽을 추론하는 확률 모델 알고리즘
- pyLDAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt, ratings_train.txt, ratings_test.txt 다운로드

- id(리뷰 번호), document(리뷰 내용), label(감성 분류 클래스, 0: 부정 / 1: 긍정)

In [1]:
# 필요 라이브러리 사용 등록
import pandas as pd
import numpy as np
import re

In [2]:
# 모든 모듈/라이브러리 버전 확인은 __version__
pd.__version__

'2.2.1'

In [3]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : bdc79c146c2e32f2cab629be240f01658cfb6cc2
python                : 3.12.2.final.0
python-bits           : 64
OS                    : Windows
OS-release            : 10
Version               : 10.0.19045
machine               : AMD64
processor             : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
byteorder             : little
LC_ALL                : None
LANG                  : None
LOCALE                : Korean_Korea.949

pandas                : 2.2.1
numpy                 : 1.26.4
pytz                  : 2024.1
dateutil              : 2.8.2
setuptools            : 69.1.1
pip                   : 24.0
Cython                : None
pytest                : None
hypothesis            : None
sphinx                : None
blosc                 : None
feather               : None
xlsxwriter            : None
lxml.etree            : 5.1.0
html5lib              : None
pymysql               : None
psycopg2              : Non

In [4]:
# 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [5]:
dfNsmcTrain.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
# 결측치 확인
dfNsmcTrain[dfNsmcTrain['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [8]:
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [9]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [10]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [11]:
# Regular Expression(정규식)으로 한글 이외의 문제는 다 제거
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+', ' ', x))

In [12]:
dfNsmcTrain[10:30]

,id,document,label
10,9008700,걍인피니트가짱이다 진짜짱이다,1
11,10217543,볼때마다 눈물나서 죽겠다 년대의 향수자극 허진호는 감성절제멜로의 달인이다,1
12,5957425,울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해,0
13,8628627,담백하고 깔끔해서 좋다 신문기사로만 보다 보면 자꾸 잊어버린다 그들도 사람이었다는 것을,1
14,9864035,취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어...,0
15,6852435,ㄱ냥 매번 긴장되고 재밋음,1
16,9143163,참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다 그...,1
17,4891476,굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐,0
18,7465483,이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드,1
19,3989148,약탈자를 위한 변명 이라 저놈들은 착한놈들 절대 아닌걸요,1


In [13]:
# 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep='\t', encoding='utf-8')

In [14]:
dfNsmcTest.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [15]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [16]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [17]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'].notnull()]

In [18]:
dfNsmcTest['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [19]:
# Regular Expression(정규식)으로 한글 이외의 문제는 다 제거
dfNsmcTest['document'] = dfNsmcTest['document'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+', ' ', x))

In [20]:
# 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터 제거
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'] != ' ']

In [21]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49443 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49443 non-null  int64 
 1   document  49443 non-null  object
 2   label     49443 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


##### 분석모델 구축

- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한글 형태소 분석은 konlpy 패키의 Okt 클래스 사용

##### KoNLPy 라이브러리 설치
- JDK 설치
- 시스템 등록 정보 JAVA_HOME 설정
    - 프로그램 실행: sysdm.cpl
    - 시스템 속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템 변수 > 새로 만들기 버튼 클릭
    - JAVA_HOME과 경로 추가 등록
- JPype1 
    - 파이썬에서 JDK를 사용하게 해주는 프로그램

In [22]:
# Jpype1 설치
!pip install JPype1

In [23]:
# koNLPy 설치
!pip install koNLPy

In [28]:
# 특성 벡터화 작업
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
okt = Okt()

In [26]:
def oktTokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [29]:
tfidf = TfidfVectorizer(tokenizer=oktTokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)

In [30]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

c:\DEV\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [31]:
nsmc_train_tfidf

<149995x113939 sparse matrix of type '<class 'numpy.float64'>'
	with 2681545 stored elements in Compressed Sparse Row format>

In [ ]:
with open('result.npy', 'wb') as f:
    np.save(f, nsmc_train_tfidf)